In [14]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    
    
class Order:
    """
    基础订单
    """
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = "<Order total : {:.2f} due: {:.2f}>"
        return fmt.format(self.total(), self.due())
    

class Promotion(ABC):
    """
    策略
    """
    @abstractmethod
    def discount(self, order):
        """返回折扣"""
        

class FidelityPromo(Promotion):
    """
    第一个具体策略
    为积分为1000或以上的顾客提供5%折扣
    """
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
    

class BulkItemPromo(Promotion):
    """
    第二个具体的策略
    单个商品为20个或以上时提供10%折扣
    """
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount
    
    
class LargeOrderPromo(Promotion):
    """
    第三个具体策略
    订单中的不同商品达到10种以上 %7
    """
    def discount(self, order):
        discount_items = {item.product for item in order.cart}
        if len(discount_items) >= 10:
            return order.total() * 0.07
        return 0

In [9]:
joe = Customer('joe', 0)
ann = Customer('ann', 1100)

In [16]:
cart = [LineItem('banana', 4, 0.5),LineItem('apple', 10, 1.5),LineItem('water', 5, 5.0)]

In [17]:
Order(joe, cart, FidelityPromo())

<Order total : 42.00 due: 42.00>

In [18]:
Order(ann, cart, FidelityPromo())

<Order total : 42.00 due: 39.90>

In [20]:
Order(joe, cart, BulkItemPromo()), Order(ann, cart, BulkItemPromo())

(<Order total : 42.00 due: 42.00>, <Order total : 42.00 due: 42.00>)